In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Imports

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image

import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, models, transforms
from torch.utils.data import DataLoader

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

torch.manual_seed(42)
if device:
    torch.cuda.manual_seed(42)

# Load Data

In [ ]:
root_dir = '/kaggle/input/hymenoptera-data/hymenoptera_data/hymenoptera_data/'

In [ ]:
image_paths = []
labels = []

for folder in os.listdir(root_dir):
    for label in os.listdir(os.path.join(root_dir, folder)):
        for file in os.listdir(os.path.join(root_dir, folder, label)):
            image_paths.append(os.path.join(root_dir, folder, label, file))
            labels.append(str(label))

In [ ]:
df = pd.DataFrame({
    'image_path': image_paths,
    'label': labels
})
df = df.sample(frac=1).reset_index(drop=True)
df.head()

# Visualization

In [ ]:
def visualize_df(df: pd.DataFrame):
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))

    for i, ax in enumerate(axes.ravel()):
        if i < len(df):
            a = np.random.randint(1, len(df), 1)[0]
            img_path = df.loc[a][['image_path']].values[0]
            label = df.loc[a][['label']].values[0]
            
            image = Image.open(img_path).convert('RGB')
            
            ax.imshow(image)
            ax.set_title(f"Label: {label}")
            ax.axis('off')
            
        else:
            ax.axis('off')
            
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_df(df)

# Preprocess

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.490, 0.449, 0.411],
        [0.231, 0.221, 0.230]
    )
])

In [ ]:
train_data = datasets.ImageFolder(
    root=os.path.join(root_dir, 'train'),
    transform=train_transforms
)

In [ ]:
train_loader = DataLoader(
    dataset=train_data,
    batch_size=8,
    shuffle=True
)

In [ ]:
valid_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(
        [0.490, 0.449, 0.411],
        [0.231, 0.221, 0.230]
    )
])

In [ ]:
valid_data = datasets.ImageFolder(
    root=os.path.join(root_dir, 'val'),
    transform=valid_transforms
)

In [ ]:
valid_loader = DataLoader(
    dataset=valid_data,
    batch_size=8,
    shuffle=True
)

In [ ]:
classes = train_data.classes
print(classes)

# Model

In [ ]:
class VGG16(nn.Module):
    def __init__(self):
        super().__init__()
        self.vgg16 = torchvision.models.vgg16(weights='DEFAULT')
        self.vgg16.classifier[6] = nn.Linear(4096, 2)

    def forward(self, x):
        return self.vgg16(x)

In [ ]:
model = VGG16()
model.train()
model = model.to(device)

# Train

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [ ]:
losses = []

In [ ]:
for epoch in range(10):
    curr_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs = data[0].to(device)
        labels = data[1].to(device)

        optimizer.zero_grad()

        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        curr_loss += loss.item()
        if i % 100 == 0:
            print(f'Epoch [{epoch+1} / 10] - Loss: {curr_loss / 100:.3f}')

    losses.append(curr_loss / 100)

# Results

In [ ]:
plt.figure(figsize=(8, 6))
plt.plot(losses)
plt.title('Loss Curve')
plt.show()

In [ ]:
correct_pred = {class_name: 0 for class_name in classes}
total_pred = {class_name: 0 for class_name in classes}

with torch.no_grad():
    model.eval()

    for data in valid_loader:
        inputs = data[0].to(device)
        labels = data[1].to(device)

        outputs = model(inputs)
        _, predictions = torch.max(outputs, 1)
        for label, prediction in zip(labels, predictions):
            if label == prediction:
                correct_pred[classes[label]] += 1
            total_pred[classes[label]] += 1

for class_name, correct_count in correct_pred.items():
    accuracy = 100 * float(correct_count) / total_pred[class_name]
    print(f'Accuracy for class: {class_name} is {accuracy:.2f} %')

total_accuracy = 100 * float(np.sum(list(correct_pred.values())) / np.sum(list(total_pred.values())))
print(f'Total Accuracy: {total_accuracy:.2f} %')

# Test

In [ ]:
def show_image(image):
    image = image.numpy().transpose((1, 2, 0))
    imagenet_avg = np.array([0.490, 0.449, 0.411])
    imagenet_std = np.array([0.231, 0.221, 0.230])
    image = imagenet_std * image + imagenet_avg
    image = np.clip(image, 0, 1)
    return image

In [ ]:
def visualize_predictions(model, data_loader):
    model.eval()
    
    fig, axes = plt.subplots(4, 4, figsize=(12, 12))
    axes = axes.flatten()
    counter = 0
    
    with torch.no_grad():
        for images, labels in data_loader:
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            _, predictions = torch.max(outputs, 1)
            
            for j in range(images.size(0)):
                if counter >= len(axes):
                    break
                
                ax = axes[counter]
                ax.imshow(show_image(images.cpu().data[j]))
                color = 'green' if classes[predictions[j]] == classes[labels[j]] else 'red'
                ax.set_title(f'True: {classes[labels[j]]}\nPred: {classes[predictions[j]]}', color=color)
                ax.axis('off')
                counter += 1
            
            if counter >= len(axes):
                break
    
    plt.tight_layout()
    plt.show()

In [ ]:
visualize_predictions(model, valid_loader)